In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..','..','..'))
from configs import spark_config as config
from utils import spark_utils as utils

spark = utils.create_spark_session('merge_team_stats')

directory = config.PROCESSED_DATA_DIR

matches = spark.read.csv(os.path.join(directory, 'matches.csv'), header=True, inferSchema=True).withColumnRenamed('season', 'Season')
teams = spark.read.csv(os.path.join(directory, 'team_stats.csv'), header=True, inferSchema=True)

teams.show(5)
matches.show(5)

/home/ravikumar/miniconda3/envs/t20i/lib/python3.12/site-packages/airflow/configuration.py:859 FutureWarning: section/key [core/sql_alchemy_conn] has been deprecated, you should use[database/sql_alchemy_conn] instead. Please update your `conf.get*` call to use the new name

[2024-11-24T14:32:12.057+0530] {spark_utils.py:17} INFO - Creating Spark session.


your 131072x1 screen size is bogus. expect trouble
24/11/24 14:32:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[2024-11-24T14:32:15.533+0530] {spark_utils.py:37} INFO - Spark session created successfully.


+-----------+-------+--------------+---------------+---------------+-------------+--------------+-----------------+-----------------+
|       Team| Season|Cumulative Won|Cumulative Lost|Cumulative Tied|Cumulative NR|Cumulative W/L|Cumulative AveRPW|Cumulative AveRPO|
+-----------+-------+--------------+---------------+---------------+-------------+--------------+-----------------+-----------------+
|Afghanistan|2009/10|             0|              0|              0|            0|           0.0|              0.0|              0.0|
|Afghanistan|   2010|             4|              2|              0|            0|           2.0|            20.22|             6.89|
|Afghanistan|2011/12|             4|              4|              0|            0|           1.0|            17.87|             6.52|
|Afghanistan|2012/13|             6|              5|              0|            0|           1.2|            19.18|             6.92|
|Afghanistan|2013/14|             8|              7|          

In [2]:
tdt = teams.select("Team").distinct().collect()
mdt = matches.select("winner").distinct().collect()

In [3]:
for i in tdt:
    if i not in mdt:
        print(i)

Row(Team='Czech Rep.')
Row(Team='Afghanistan')
Row(Team='Ivory Coast')
Row(Team='P.N.G.')
Row(Team='Peru')
Row(Team='Chile')
Row(Team='Cayman')
Row(Team='Iran')
Row(Team='U.S.A.')
Row(Team='Swaziland')
Row(Team='World-XI')
Row(Team='U.A.E.')
Row(Team='South Korea')
Row(Team='Mongolia')
Row(Team='Slovenia')
Row(Team='Mali')


In [4]:
for i in mdt:
    if i not in tdt:
        print(i)

Row(winner='Cayman Islands')
Row(winner='Barbados')
Row(winner='United States of America')
Row(winner='United Arab Emirates')
Row(winner='Czech Republic')
Row(winner='Papua New Guinea')


In [5]:
# Teams that need to be mapped between `tdt` and `mdt`
team_name_mapping = {
    'U.S.A.': 'United States of America',
    'U.A.E.': 'United Arab Emirates',
    'Czech Rep.': 'Czech Republic',
    'P.N.G.': 'Papua New Guinea',
    'Cayman': 'Cayman Islands'
}

# Teams that do not have a direct match
unmatched_tdt = [team["Team"] for team in tdt if team not in mdt and team not in team_name_mapping]
unmatched_mdt = [team["winner"] for team in mdt if team not in tdt and team not in team_name_mapping.values()]

print("Mapped Team Names Dictionary:", team_name_mapping)
print("Unmatched Teams in tdt:", unmatched_tdt)
print("Unmatched Teams in mdt:", unmatched_mdt)

Mapped Team Names Dictionary: {'U.S.A.': 'United States of America', 'U.A.E.': 'United Arab Emirates', 'Czech Rep.': 'Czech Republic', 'P.N.G.': 'Papua New Guinea', 'Cayman': 'Cayman Islands'}
Unmatched Teams in tdt: ['Czech Rep.', 'Afghanistan', 'Ivory Coast', 'P.N.G.', 'Peru', 'Chile', 'Cayman', 'Iran', 'U.S.A.', 'Swaziland', 'World-XI', 'U.A.E.', 'South Korea', 'Mongolia', 'Slovenia', 'Mali']
Unmatched Teams in mdt: ['Cayman Islands', 'Barbados', 'United States of America', 'United Arab Emirates', 'Czech Republic', 'Papua New Guinea']


In [6]:
unmatched_teams = unmatched_tdt + unmatched_mdt
unmatched_teams

['Czech Rep.',
 'Afghanistan',
 'Ivory Coast',
 'P.N.G.',
 'Peru',
 'Chile',
 'Cayman',
 'Iran',
 'U.S.A.',
 'Swaziland',
 'World-XI',
 'U.A.E.',
 'South Korea',
 'Mongolia',
 'Slovenia',
 'Mali',
 'Cayman Islands',
 'Barbados',
 'United States of America',
 'United Arab Emirates',
 'Czech Republic',
 'Papua New Guinea']

In [7]:
print(teams.count(), matches.count())
teams = teams.filter(~teams['Team'].isin(unmatched_teams))
matches = matches.filter(~matches['team1'].isin(unmatched_teams)).filter(~matches['team2'].isin(unmatched_teams))
print(teams.count(), matches.count())

924 3710
844 3306


In [8]:
teams = teams.replace(team_name_mapping, subset='Team')
matches = matches.replace(team_name_mapping, subset='team1').replace(team_name_mapping, subset='team2').replace(team_name_mapping, subset='winner')

In [9]:
matches.show(5)
teams.show(5)

+--------+-----------+----------+------+-------+-----------+
|match_id|      team1|     team2|gender| Season|     winner|
+--------+-----------+----------+------+-------+-----------+
| 1019981|New Zealand|Bangladesh|  male|2016/17|New Zealand|
| 1031665|West Indies|   England|  male|   2017|West Indies|
| 1001349|  Australia| Sri Lanka|  male|2016/17|  Sri Lanka|
| 1034825|      India|   England|  male|2016/17|    England|
| 1019979|New Zealand|Bangladesh|  male|2016/17|New Zealand|
+--------+-----------+----------+------+-------+-----------+
only showing top 5 rows

+---------+-------+--------------+---------------+---------------+-------------+--------------+-----------------+-----------------+
|     Team| Season|Cumulative Won|Cumulative Lost|Cumulative Tied|Cumulative NR|Cumulative W/L|Cumulative AveRPW|Cumulative AveRPO|
+---------+-------+--------------+---------------+---------------+-------------+--------------+-----------------+-----------------+
|Argentina|2019/20|           

In [10]:
matches = matches.withColumnRenamed("Season","Match_Season")
matches = matches.join(teams, on=[matches['team1'] == teams['Team'], matches['Match_Season'] == teams['Season']], how='inner').drop("Team","Season")
matches.show(5)

+--------+-----------+----------+------+------------+-----------+--------------+---------------+---------------+-------------+--------------+-----------------+-----------------+
|match_id|      team1|     team2|gender|Match_Season|     winner|Cumulative Won|Cumulative Lost|Cumulative Tied|Cumulative NR|Cumulative W/L|Cumulative AveRPW|Cumulative AveRPO|
+--------+-----------+----------+------+------------+-----------+--------------+---------------+---------------+-------------+--------------+-----------------+-----------------+
| 1019981|New Zealand|Bangladesh|  male|     2016/17|New Zealand|            46|             40|              5|            2|          1.15|            24.19|              7.9|
| 1031665|West Indies|   England|  male|        2017|West Indies|            39|             37|              3|            3|          1.05|            23.76|             7.78|
| 1001349|  Australia| Sri Lanka|  male|     2016/17|  Sri Lanka|            46|             41|              

In [11]:
for old_name, new_name in {
    "Cumulative Won": "Cumulative Won team1",
    "Cumulative Lost": "Cumulative Lost team1",
    "Cumulative Tied": "Cumulative Tied team1",
    "Cumulative W/L": "Cumulative W/L team1",
    "Cumulative AveRPW": "Cumulative AveRPW team1", 
    "Cumulative AveRPO": "Cumulative AveRPO team1", 
}.items():
    matches = matches.withColumnRenamed(old_name, new_name)

matches = matches.join(teams, on=[matches['team2'] == teams['Team'], matches['Match_Season'] == teams['Season']], how='inner').drop("Team","Match_Season")

for old_name, new_name in {
    "Cumulative Won": "Cumulative Won team2",
    "Cumulative Lost": "Cumulative Lost team2",
    "Cumulative Tied": "Cumulative Tied team2",
    "Cumulative W/L": "Cumulative W/L team2",
    "Cumulative AveRPW": "Cumulative AveRPW team2",
    "Cumulative AveRPO": "Cumulative AveRPO team2",
}.items():
    matches = matches.withColumnRenamed(old_name, new_name)

matches.show(5)

+--------+-----------+----------+------+-----------+--------------------+---------------------+---------------------+-------------+--------------------+-----------------------+-----------------------+-------+--------------------+---------------------+---------------------+-------------+--------------------+-----------------------+-----------------------+
|match_id|      team1|     team2|gender|     winner|Cumulative Won team1|Cumulative Lost team1|Cumulative Tied team1|Cumulative NR|Cumulative W/L team1|Cumulative AveRPW team1|Cumulative AveRPO team1| Season|Cumulative Won team2|Cumulative Lost team2|Cumulative Tied team2|Cumulative NR|Cumulative W/L team2|Cumulative AveRPW team2|Cumulative AveRPO team2|
+--------+-----------+----------+------+-----------+--------------------+---------------------+---------------------+-------------+--------------------+-----------------------+-----------------------+-------+--------------------+---------------------+---------------------+-------------

In [12]:
# male 0 female 1
from pyspark.sql import functions as F
matches = matches.withColumn("gender", F.when(matches['gender']=="male",0).otherwise(1).cast("int"))
matches.show(5)

+--------+-----------+----------+------+-----------+--------------------+---------------------+---------------------+-------------+--------------------+-----------------------+-----------------------+-------+--------------------+---------------------+---------------------+-------------+--------------------+-----------------------+-----------------------+
|match_id|      team1|     team2|gender|     winner|Cumulative Won team1|Cumulative Lost team1|Cumulative Tied team1|Cumulative NR|Cumulative W/L team1|Cumulative AveRPW team1|Cumulative AveRPO team1| Season|Cumulative Won team2|Cumulative Lost team2|Cumulative Tied team2|Cumulative NR|Cumulative W/L team2|Cumulative AveRPW team2|Cumulative AveRPO team2|
+--------+-----------+----------+------+-----------+--------------------+---------------------+---------------------+-------------+--------------------+-----------------------+-----------------------+-------+--------------------+---------------------+---------------------+-------------

In [13]:
matches = matches.select("match_id","gender","Cumulative Won team1","Cumulative Lost team1","Cumulative Tied team1","Cumulative W/L team1","Cumulative AveRPW team1","Cumulative AveRPO team1","Cumulative Won team2","Cumulative Lost team2","Cumulative Tied team2","Cumulative W/L team2","Cumulative AveRPW team2","Cumulative AveRPO team2").sort("match_id")
matches.show(5)
matches.count()

+--------+------+--------------------+---------------------+---------------------+--------------------+-----------------------+-----------------------+--------------------+---------------------+---------------------+--------------------+-----------------------+-----------------------+
|match_id|gender|Cumulative Won team1|Cumulative Lost team1|Cumulative Tied team1|Cumulative W/L team1|Cumulative AveRPW team1|Cumulative AveRPO team1|Cumulative Won team2|Cumulative Lost team2|Cumulative Tied team2|Cumulative W/L team2|Cumulative AveRPW team2|Cumulative AveRPO team2|
+--------+------+--------------------+---------------------+---------------------+--------------------+-----------------------+-----------------------+--------------------+---------------------+---------------------+--------------------+-----------------------+-----------------------+
|  211028|     0|                   0|                    0|                    0|                 0.0|                    0.0|               

3028

In [14]:
utils.spark_save_data(matches,config.MERGED_DATA_DIR, "team_stats.csv")
spark.stop()

[2024-11-24T14:34:03.773+0530] {spark_utils.py:64} INFO - Successfully wrote data to /usr/ravi/t20/data/3_mergedData/team_stats.csv
